In [13]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology
import neuroml
import arbor
import re
import pyNN.morphology as pyNNmorph

In [14]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))
nu_morph = NeuroMLMorphology(neuroml_morph)

In [17]:
class ArborTemplate(object):

    #def __init__(self, morphology, cm, Ra, **other_parameters):
    def __init__(self, morphology, **other_parameters):
        import neuroml
        import neuroml.arraymorph

        #self.traces = defaultdict(list)
        #self.recording_time = False
        #self.spike_source = None
        # self.spike_times = h.Vector(0)  # change this for arbor

        # create morphology
        self.morphology = morphology
        # self.sections = {} # is this needed for arbor?
        # self.section_labels = {} # dito

        # create morphology
        self.sections = self._create_arbor_tree()  # arbor segment tree ~ NEURON section i.e unbranched collection of segments
        self.section_labels = arbor.label_dict(self._create_region_definitions())
        # create cell
        self.cell = arbor.cable_cell(self.sections, self.section_labels)
        # set cell properties
        #self.cell.set_properties(cm=cm, rL=Ra)
        # setup ion, setup channel mechanism and attach the mechanism
        #self.setup_ionic_species()
        #self.setup_channel_mechanism(other_parameters)
        #self.attach_mechanisms(other_parameters)

        # create morphology
        self._create_arbor_morphology()  # this creates self.__arbor_morphology
        self._create_arbor_labels()  # this creates self.__arbor_labels
        # Create decorations
        self._decor = arbor.decor()
        if "ionic_species" in other_parameters.keys():
            self._decorate_ionic_species(other_parameters)  # updates self._decor
        self._decorate_channels(other_parameters)  # updates self._decor

        for receptor_name in self.post_synaptic_entities:
            self.morphology.synaptic_receptors[receptor_name] = defaultdict(list)

        for name, pse in self.post_synaptic_entities.items():
            parameters = other_parameters[name]
            mechanism_name = pse.model
            synapse_model = getattr(h, mechanism_name)
            density_function = parameters["density"]
            for index, id in enumerate(self.sections):
                density = density_function.value_in(self.morphology, index)
                if density > 0:
                    n_synapses, remainder = divmod(density, 1)
                    rnd = numpy.random  # todo: use the RNG from the parent Population
                    if rnd.uniform() < remainder:
                        n_synapses += 1
                    section = self.sections[id]
                    for i in range(int(n_synapses)):
                        self.morphology.synaptic_receptors[name][id].append(synapse_model(0.5, sec=section))

        # handle ionic species
        def set_in_section(section, index, name, value):
            if isinstance(value, IonChannelDistribution):  # should be "NeuriteDistribution"
                value = value.value_in(self.morphology, index)
            if value is not None:
                if name == "eca":  # tmp hack
                    section.push()
                    h.ion_style("ca_ion", 1, 1, 0, 1, 0)
                    h.pop_section()
                try:
                    setattr(section, name, value)
                except (NameError, AttributeError) as err:  # section does not contain ion
                    if "the mechanism does not exist" not in str(err):
                        raise

        for ion_name, parameters in self.ionic_species.items():
            for index, id in enumerate(self.sections):
                section = self.sections[id]
                set_in_section(section, index, "e{}".format(ion_name), parameters.reversal_potential)
                if parameters.internal_concentration:
                    set_in_section(section, index, "{}i".format(ion_name), parameters.internal_concentration)
                if parameters.external_concentration:
                    set_in_section(section, index, "{}o".format(ion_name), parameters.external_concentration)

        # set source section
        if self.spike_source:
            self.source_section = self.sections[self.spike_source]
        elif "axon_initial_segment" in self.sections:
            self.source_section = self.sections["axon_initial_segment"]
        else:
            self.source_section = self.sections[morphology.soma_index]
        self.source = self.source_section(0.5)._ref_v
        self.rec = h.NetCon(self.source, None, sec=self.source_section)

    ######################################
    # Functions to create Arbor morphology
    ######################################
    def _create_arbor_morphology(self):
        """
        Call this as self._create_arbor_morphology() to create self.__arbor_morphology
        """
        self.__arbor_morphology = arbor.morphology(self._create_arbor_tree())

    def _create_arbor_tree(self):
        # Creates tree directly from file if available.
        # if hasattr(self.morphology, "local_file"):
        #     tree = arbor.load_swc_arbor(self.morphology.local_file)
        # else:
        #     tree = arbor.segment_tree()
        #     for i, nml_seg in enumerate(self.morphology.segments):
        #         self._append_arbor_tree(tree, nml_seg)
        tree = arbor.segment_tree()
        for i, nml_seg in enumerate(self.morphology.segments):
            self._append_arbor_tree(tree, nml_seg)
        return tree  # arbor segment tree ~ NEURON section i.e unbranched collection of segments

    def _append_arbor_tree(self, tree, nml_seg):
        if not nml_seg.parent:
            tree.append(arbor.mnpos,
                        arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                     nml_seg.proximal.diameter / 2),
                        arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                     nml_seg.distal.diameter / 2), tag=self._get_swc_tag(nml_seg))
        else:
            tree.append(nml_seg.parent.id,
                        arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                     nml_seg.proximal.diameter / 2),
                        arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                     nml_seg.distal.diameter / 2), tag=self._get_swc_tag(nml_seg))

    ########### END OF FUNCTION ##############

    ##########################################
    # Functions to create label for cable cell
    ##########################################
    def _create_arbor_labels(self):
        """
        Call this as self._create_arbor_labels() to create self.__arbor_labels
        """
        self.__arbor_labels = arbor.label_dict(self._create_region_definitions())

    def _create_region_definitions(self):
        # Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
        dict_defs = {}
        for indx, nml_seg in enumerate(self.morphology.segments):
            if nml_seg.name is None:  # although indices in the swc file may begin with 1
                self._add_segment_name(indx, nml_seg)  # indices in morphology.segments start from 0
            dict_defs.update({nml_seg.name: "(tag " + str(self._get_swc_tag(nml_seg)) + ")"})
        dict_defs.update({"everywhere": "(all)"})
        return dict_defs

    def _get_swc_tag(self, nml_seg):
        if re.search("soma", nml_seg.name, re.IGNORECASE):
            return pyNNmorph.SOMA  # = 1
        elif re.search("axon", nml_seg.name, re.IGNORECASE):
            return pyNNmorph.AXON  # = 2
        elif (re.search("basal", nml_seg.name, re.IGNORECASE)
              and re.search("dend", nml_seg.name, re.IGNORECASE)):
            return pyNNmorph.BASALDENDRITE  # = 3
        elif (re.search("apical", nml_seg.name, re.IGNORECASE)
              and re.search("dend", nml_seg.name, re.IGNORECASE)):
            return pyNNmorph.APICALDENDRITE  # = 4
        elif re.search("custom", nml_seg.name, re.IGNORECASE):
            return pyNNmorph.CUSTOM  # = 5
        elif re.search("neurite", nml_seg.name, re.IGNORECASE):
            return pyNNmorph.UNSPECIFIEDNEURITES  # = 6
        elif re.search("glia", nml_seg.name, re.IGNORECASE):
            return pyNNmorph.GLIAPROCESSES  # = 7
        else:
            return pyNNmorph.UNDEFINED  # = 0

    def _add_segment_name(self, indx, nml_seg):
        # Consider moving self._add_segment_name(indx, nml_seg), this method
        # and the _get_name method in NeuroMLMorphology class of pyNN.morphology file
        for swc_tag, array_morph in self.morphology.section_groups.items():
            if indx in array_morph:
                setattr(nml_seg, "name", self._get_name(swc_tag))
                # nml_seg.name = self._get_name(swc_tag)
        if nml_seg.name is None:  # if it is still None, then it is soma segment i.e. swc tag = 1
            setattr(nml_seg, "name", self._get_name(1))
            # nml_seg.name = self._get_name(1)
        # return nml_seg

    def _get_name(self, swc_tag):
        tag_and_name = {
            0: "UNDEFINED",
            1: "soma",
            2: "axon",
            3: "basal_dendrite",
            4: "apical_dendrite",
            5: "CUSTOM",
            6: "unspecific_neurite",
            7: "glia",
        }
        return tag_and_name[swc_tag]

    ########### END OF FUNCTION ##############

    ########################################
    # Functions to decorate ions in the cell
    ########################################
    def _decorate_ionic_species(self, other_parameters):
        """
        Cal this as self._decorate_ionic_species(other_parameters) to update self._decor
        """
        translated_keys = {"external_concentration": "ext_con",
                           "internal_concentration": "int_con",
                           "ion_name": "ion_name",
                           "reversal_potential": "rev_pot"}
        for ion_name, ion_sp in other_parameters["ionic_species"].item():
            dict_input = self._get_dict_for_ion(translated_keys, ion_sp)
            self._decor.set_ion(getattr(ion_sp, "ion_name"), **dict_input)

    def _get_dict_for_ion(self, translated_keys, the_ionic_species):
        keys_in_the_ionic_species = [a for a in dir(the_ionic_species) if not a.startswith('__')]
        dict_for_ion = {}
        for key in keys_in_the_ionic_species:
            if not getattr(the_ionic_species, key):
                pass  # empty so do nothing
            else:
                if key == "ion_name":
                    dict_for_ion.update({"method": "nernst/" + getattr(the_ionic_species, key)})
                else:
                    dict_for_ion.update({translated_keys[key]: getattr(the_ionic_species, key)})
        return dict_for_ion

    ######################################

    ###############################
    # Functions to decorate channel
    ###############################
    def _decorate_channels(self, other_parameters):
        """
        Call this as self._decorate_channels(other_parameters) to update self._decor.
        Decoration in this context means painting channel mechanisms on respective
        regions of self.__arbor_morphology
        """
        arbor_channel_mechanisms = self._create_arbor_channel_mechanisms(other_parameters)
        for chnl_name, ion_chnl in self.ion_channels.items():
            if ion_chnl.model in arbor_channel_mechanisms.keys():
                paint_site = self._format_extracted_channel_region(chnl_name, other_parameters)
                self._decor.paint(paint_site, arbor_channel_mechanisms[ion_chnl.model])

    def _format_extracted_channel_region(self, channel_name, other_parameters):
        region = other_parameters[channel_name]["conductance_density"].selector
        return '"{}"'.format(region)  # In the form '"region"'

    def _create_arbor_channel_mechanisms(self, other_parameters):
        # Returns arbor's channel mechanisms as a values of keys in the dictionary
        # Dictionary keys corresponds to respective model name associated with an ion channel.
        get_corrected_mechanism_name_for_pas = lambda mech_name: \
            [mech_name if (mech_name != "pas") else "pas/e=" + other_parameters["pas"]["e_rev"]][0]  # 'pas/e=-45'
        channel_mechanisms = {}
        for channel_name, ion_channel in self.ion_channels.items():
            mechanism_name = ion_channel.model
            its_param_dict = self._create_mechanism_parameters_dict(mechanism_name, other_parameters)
            channel_mechanisms.update(
                {mechanism_name: arbor.mechanism(
                    get_corrected_mechanism_name_for_pas(mechanism_name), its_param_dict)})
        # self.__arbor_channel_mechanisms = channel_mechanisms
        return channel_mechanisms

    def _create_mechanism_parameters_dict(self, mechanism_name, other_parameters):
        # For desired mechanism return dictionary, say,
        # {'g': 0.0003} OR
        # {"ena": 50., "gnabar": 0.120, "ek": -77., "gkbar": 0.036, "gl": 0.}
        # CURRENLY SUPPORTED MECHANISMS: "pas" and "hh".
        get_dict_for_pas = lambda chnnl_type, param_key, param_value: \
            {chnnl_type.conductance_density_parameter: param_value.value} if (param_key == "conductance_density") \
                else {channl_type.translations[param_key]["translated_name"]: param_value}
        mech_param_dict = {}
        if mechanism_name == "pas":
            channel_type = self.ion_channels["pas"]
            for param_name, its_value in other_parameters["pas"].items():
                mech_param_dict.update(get_dict_for_pas(channel_type, param_name, its_value))
        elif mechanism_name == "hh":
            for channel_name, ion_channel in self.ion_channels.items():
                if ion_channel.model == "hh":
                    channel_type = self.ion_channels[channel_name]
                    for param_name, its_value in other_parameters[channel_name].items():
                        mech_param_dict.update(
                            {channel_type.conductance_density_parameter: its_value.value})
            mech_param_dict.update({'gl': 0})  # avoids duplication with pas THIS MAY NOT BE NEEDED FOR FUTURE Arbor
        return mech_param_dict

    ########### END OF FUNCTION ##############

    ######################################
    # Function to insert channel mechanisms
    ######################################
    def attach_mechanism(self, other_parameters):
        for mechanism_name, the_mechanism in self.mechanisms.items():
            for channel_name, ion_channel in self.ion_channels.items():
                if ion_channel.model == mechanism_name:
                    region = other_parameters[channel_name]["conductance_density"].selector
                    for chnnl_name, ion_chnnl in self.ion_channels.items():
                        param_region = other_parameters[chnnl_name]["conductance_density"].selector
                        if param_region == region:
                            if "e_rev" in other_parameters[chnnl_name].keys():
                                self.cell.paint('"{}"'.format(region), chnnl_name,
                                                rev_pot=other_parameters[chnnl_name]["e_rev"])
                    self.cell.paint('"{}"'.format(region), the_mechanism)

    ######################################

    ###############################
    # Functions to decorate synapse
    ###############################
    def _append_synapse_labels(self):
        """
        Call this as self._append_synapse_labels() to append synapse labels into self.__arbor_labels
        """
        self.__arbor_labels.append(arbor.label_dict(self._create_synapse_definitions()))

    def _create_synapse_definitions(self, other_parameters):
        # Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
        dict_defs = {}
        for name, pse in self.post_synaptic_entities.items():
            region_name = self._get_pse_region_name(name, other_parameters)
            synapse_site = "synapse_" + region_name
            if re.search("soma", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            elif re.search("axon", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            elif re.search("basal", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            elif (re.search("apical", region_name, re.IGNORECASE)
                  and re.search("dend", region_name, re.IGNORECASE)):
                dict_defs.update({synapse_site: ""})
            elif re.search("custom", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            elif re.search("neurite", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            elif re.search("glia", region_name, re.IGNORECASE):
                dict_defs.update({synapse_site: ""})
            else:
                dict_defs.update({synapse_site: "(on-branches 0.5)"})  # DEFAULT is mid-point of all branches
                # Another choice is uniformly distributed random locations, for 10 random locations
                # "(uniform (all) 0 9 0)" last integer (here zero) is seed number.
                # "(on-branches 0.5)" is the default because `from pyNN.morphology import uniform`
                # does not mean uniform distribution, instead "uniform" in the context of density
                # should be thought of as homegeneity since the synapse are homogeneously distributed
                # across all branches or every branch of a specific tag (say, dendrites) such that
                # the synapse location is given by the second argument of the density function
                # (here, uniform function).
        return dict_defs

    # AMPA={"density": uniform('all', 0.05),  # number per µm
    #       "e_rev": 0.0,
    #       "tau_syn": 2.0},
    # GABA_A={"density": by_distance(dendrites(), lambda d: 0.05 * (d < 50.0)),  # number per µm
    #         "e_rev": -70.0,
    #         "tau_syn": 5.0}
    def _get_pse_region_name(self, pse_name, other_parameters):
        pse_density_site = other_parameters[pse_name]["density"].selector
        if isinstance(pse_density_site, str):
            return pse_density_site
        else:
            return pse_density_site.__class__.__name__

    ########### END OF FUNCTION ##############

    def memb_init(self):
        for state_var in ('v',):
            initial_value = getattr(self, '{0}_init'.format(state_var))
            assert initial_value is not None
            if state_var == 'v':
                for section in self.sections.values():
                    for seg in section:
                        seg.v = initial_value
            else:
                raise NotImplementedError()


In [18]:
cell = ArborTemplate(nu_morph)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. arbor._arbor.cable_cell(morphology: arb::morphology, labels: arbor._arbor.label_dict, decor: arbor._arbor.decor)
    2. arbor._arbor.cable_cell(segment_tree: arb::segment_tree, labels: arbor._arbor.label_dict, decor: arbor._arbor.decor)

Invoked with: <arbor._arbor.segment_tree object at 0x7f188e0132f0>, (label_dict (region  "soma" (tag 1)) (region  "dendrite" (tag 0)) (region  "everywhere" (all)))

In [27]:
def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return pyNNmorph.SOMA  # = 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return pyNNmorph.AXON  # = 2
    elif (re.search("basal", nml_seg.name, re.IGNORECASE)
          and re.search("dend", nml_seg.name, re.IGNORECASE)):
        return pyNNmorph.BASALDENDRITE  # = 3
    elif (re.search("apical", nml_seg.name, re.IGNORECASE)
          and re.search("dend", nml_seg.name, re.IGNORECASE)):
        return pyNNmorph.APICALDENDRITE  # = 4
    elif re.search("custom", nml_seg.name, re.IGNORECASE):
        return pyNNmorph.CUSTOM  # = 5
    elif re.search("neurite", nml_seg.name, re.IGNORECASE):
        return pyNNmorph.UNSPECIFIEDNEURITES  # = 6
    elif re.search("glia", nml_seg.name, re.IGNORECASE):
        return pyNNmorph.GLIAPROCESSES  # = 7
    else:
        return pyNNmorph.UNDEFINED  # = 0

def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(arbor.mnpos,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter / 2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter / 2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter / 2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter / 2), tag=get_swc_tag(nml_seg))

def create_arbor_tree(nu_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(nu_morph.segments):
        append_arbor_tree(tree, nml_seg)
    return tree

def create_region_definitions(nu_morph):
    # Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
    dict_defs = {}
    for indx, nml_seg in enumerate(nu_morph.segments):
        if nml_seg.name is None:  # although indices in the swc file may begin with 1
            self._add_segment_name(indx, nml_seg)  # indices in morphology.segments start from 0
        dict_defs.update({nml_seg.name: "(tag " + str(get_swc_tag(nml_seg)) + ")"})
    #dict_defs.update({"everywhere": "(all)"})
    return dict_defs

def create_arbor_labels(nu_morph):
    """
    Call this as self._create_arbor_labels() to create self.__arbor_labels
    """
    arbor_labels = arbor.label_dict(create_region_definitions(nu_morph))
    return arbor_labels

In [28]:
sections = create_arbor_tree(nu_morph)  # arbor segment tree ~ NEURON section i.e unbranched collection of segments
section_labels = arbor.label_dict(create_region_definitions(nu_morph))

In [29]:
cell = arbor.cable_cell(sections, section_labels)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. arbor._arbor.cable_cell(morphology: arb::morphology, labels: arbor._arbor.label_dict, decor: arbor._arbor.decor)
    2. arbor._arbor.cable_cell(segment_tree: arb::segment_tree, labels: arbor._arbor.label_dict, decor: arbor._arbor.decor)

Invoked with: <arbor._arbor.segment_tree object at 0x7f188ba8b1f0>, (label_dict (region  "soma" (tag 1)) (region  "dendrite" (tag 0)))

In [ ]:
class ArborTemplate(object):

    #def __init__(self, morphology, cm, Ra, **other_parameters):
    def __init__(self, morphology, **other_parameters):
        import neuroml
        import neuroml.arraymorph

        # create morphology
        self.morphology = morphology
        # self.sections = {} # is this needed for arbor?
        # self.section_labels = {} # dito

        # create morphology
        self.sections = self.create_arbor_tree()  # arbor segment tree ~ NEURON section i.e unbranched collection of segments
        self.section_labels = arbor.label_dict(self._create_region_definitions())
        # create cell
        self.cell = arbor.cable_cell(self.sections, self.section_labels)

In [30]:
# (1) Create a morphology with a single (cylindrical) segment of length=diameter=6 μm
tree = arbor.segment_tree()
tree.append(arbor.mnpos, arbor.mpoint(-3, 0, 0, 3), arbor.mpoint(3, 0, 0, 3), tag=1)

# (2) Define the soma and its midpoint
labels = arbor.label_dict({'soma':   '(tag 1)',
                           'midpoint': '(location 0 0.5)'})

# (3) Create and set up a decor object
decor = arbor.decor()
decor.set_property(Vm=-40)
decor.paint('"soma"', arbor.density('hh'))
decor.place('"midpoint"', arbor.iclamp( 10, 2, 0.8), "iclamp")
decor.place('"midpoint"', arbor.spike_detector(-10), "detector")

# (4) Create cell and the single cell model based on it
cell = arbor.cable_cell(tree, labels, decor)

In [31]:
txt = "The rain in Spain"
x = re.search("^The.*Spain$", txt)

In [32]:
x

<re.Match object; span=(0, 17), match='The rain in Spain'>

In [35]:
a = "na_ion"
b = re.search(".*ion$", a)
b

<re.Match object; span=(0, 6), match='na_ion'>

In [37]:
c = {"a": 1, "b": 2, "c": 3}
for ky in c.keys():
    print(ky)

a
b
c


In [38]:
b

<re.Match object; span=(0, 6), match='na_ion'>

In [39]:
decor = arbor.decor()

In [ ]:
decor.set_ion